In [1]:
%pylab inline
import scipy as S
import emcee
from scipy import optimize as op

Populating the interactive namespace from numpy and matplotlib


In [2]:
import subprocess
import os

def compute_simu():
#calcul les points de simu s=2.5 a 157.5
    xi0_simu=zeros(16)
    xi2_simu=zeros(16)
    for i in range(0,1000):
        f =loadtxt('/home/arocher/Stage/Simu/2PCF_EBOSSCMASS/2PCF_EZmock_eBOSS_LRGpCMASS_SGC_v5_z0.6z1.0_'+(str(i+1).zfill(4))+'.dat')
        xi0_simu += f[:,1][range(4,20)]
        xi2_simu += f[:,2][range(4,20)]
    xi0_simu /= 1000
    xi2_simu /= 1000
    return xi0_simu,xi2_simu

def compute_model(f1,f2):
    run_gs =  subprocess.Popen("/home/arocher/Stage/gaussian_stream_model/code_GSRSD/GSRSD.exe %f %f"%(f1,f2), shell=True, stdout=subprocess.PIPE).stdout
    x = array(run_gs.read().decode().split(" "))
    x = delete(x,-1)              #enleve l'espace a la fin
    data = x.astype(np.float)     #lis et convertit les outputs du GS
    r=data[data%2.5==0]
    xi0 = empty(len(r))
    xi2 = empty(len(r))
    g=1
    d=2
    for i in range(len(r)):
        xi0[i] = data[g]
        xi2[i] = data[d]
        g += 3
        d += 3
    
    return xi0[range(4,20)],xi2[range(4,20)],r[range(4,20)]


xi0_simu = compute_simu()[0]
def Xi_2 (f1,f2):
    res = (compute_model(f1,f2)[0]-xi0_simu)**2
    return sum(res)


In [133]:
f1=0.3655    
out =([])
for i in range (0,20):
    print (Xi_2(f1,0),f1)
    f1 += 0.0001

8.763468566682973e-06 0.365
8.75419175869704e-06 0.3651
8.742687137103839e-06 0.36519999999999997
8.732892007091642e-06 0.36529999999999996
8.727355590320294e-06 0.36539999999999995
8.718906464279008e-06 0.36549999999999994
8.721374122997146e-06 0.3655999999999999
8.719920091801321e-06 0.3656999999999999
8.720782739691403e-06 0.3657999999999999
8.723884791995141e-06 0.3658999999999999
8.72447153517262e-06 0.3659999999999999
8.731251838081539e-06 0.36609999999999987
8.743698872170937e-06 0.36619999999999986
8.754266978150451e-06 0.36629999999999985
8.766642946954639e-06 0.36639999999999984
8.780610589260912e-06 0.3664999999999998
8.796586494010277e-06 0.3665999999999998
8.818897281598473e-06 0.3666999999999998
8.838561791680739e-06 0.3667999999999998
8.860703034447635e-06 0.3668999999999998


In [134]:
u=0
out =([])
for i in range (0,100):
    f1 += 0.01
    f2=0
    for j in range (0,200):
        f2 +=0.001
        out=append(out,[f1,f2,Xi_2(f1,f2)])
        u+=1



In [138]:
where(out == min(out))
out[14382]

0.374

In [127]:
xi0_simu = compute_simu()[0]
def Xi_2 (bias,x):
    f1,f2 = bias 
    y_model = compute_model(f1,f2)[0]
    print(f1,f2)
    return sum(y_model-x)**2
x=xi0_simu 
x0 = ([])
res = op.minimize(Xi_2, x0,args=(x))
print(res)

ValueError: not enough values to unpack (expected 2, got 0)

In [123]:
f = 0.38, 0.1
Xi_2(f)

0.38 0.1


1.6645178492757627e-05

In [59]:
#likelihood
xi0_simu = compute_simu()[0]
def lnlike(bias):
    f1,f2 = bias
    return -0.5*sum(compute_model(f1,f2)-xi0_simu)**2

def lnprior(bias):
    f1,f2 = bias
    if 0.0 < f1 < 3.0 and -1.0 < f2 < 1 :
        return 0.0
    return -np.inf

def lnprob(bias):
    lp = lnprior(bias)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(bias)

ndim, nwalkers = 2, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=())
out = sampler.run_mcmc(pos, 50)

emcee: Exception while calling your likelihood function:
  params: [0.9994707  0.10035371]
  args: ()
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/arocher/.local/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-59-19493fcc5781>", line 17, in lnprob
    return lp + lnlike(bias)
  File "<ipython-input-59-19493fcc5781>", line 5, in lnlike
    return -0.5*sum(compute_model(f1,f2)-xi0_simu)**2
  File "<ipython-input-44-2a8b7a05fd62>", line 18, in compute_model
    x = array(run_gs.read().decode().split(" "))
KeyboardInterrupt


KeyboardInterrupt: 

In [57]:
slampler

NameError: name 'slampler' is not defined

In [56]:
xl = np.array([0, 10])
for m, b, lnf in samples[np.random.randint(len(samples), size=100)]:
    pl.plot(xl, , color="k", alpha=0.1)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices